# Genetic Algorithms
## Evolutionary Algorithms for Optimization

- <a href=#NGFA>Non-Gradent Following Optimization Algorithms</a>
- <a href=#GA>Genetic Algorithm</a>
- <a href=#FSML>Feature Selection for Machine Learning</a>
- <a href=#BFDS>Best-fitting Distribution Selection</a>
- <a href=#MFM>Multivariate Function Minimization</a>

<a id=top></a>

In [ ]:
import numpy as np
import scipy.stats as stt
import pandas as pd

import chart_studio.plotly as ply
import chart_studio.tools as plytool
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as plyoff
import plotly.subplots as plysub

# to use plotly offline, need to initialize with a plot\n",
plyoff.init_notebook_mode(connected=True)
init = go.Figure(data=[go.Scatter({'x':[1, 2], 'y':[42, 42], 'mode':'markers'})], layout=go.Layout(title='Init', xaxis={'title':'x'}, height=100, width=100))
plyoff.iplot(init)

## Mathematical Optimization
Optimization is the process of finding some numerical values that generate a minimum or maximum value for a specified function. Examples include finding the best parameters for a statistical distribution fit to some data, numerically solving differential equations, or feature selection in machine learning.

There are many types and classes of optimization algorithms. Most that have been invented by mathematicianas and computer scientists rely on function derivatives / gradients. Anybody who's studied the topic even slightly will likely remember [Newton-Raphson](https://en.wikipedia.org/wiki/Newton%27s_method) or the [BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) - the latter of which even needs the second derivatives (the [Hessian](https://en.wikipedia.org/wiki/Hessian_matrix)). Mathematical optimization algorithms typically iterate over two steps:

1.  evaluate current solution
2. find new solution to try

A series of individual solutions are identified and evluated, until the sequence converges to a solution.

Gradient-following methods can have good properties, but also two major issues.

1. The first is the need to compute derivatives. This can often be very difficult - even assuming derivatives can be analytically solved - and time consuming. Furthermore, not all functions we wish to optimize even have a derivative. What is the derivative of an accuracy loss function for a [Decision Tree Classifier](https://en.wikipedia.org/wiki/Decision_tree_learning)?

2. The second issue is that gradient-followers can easily get stuck in local optima, rather than finding a global optimum. Most such algorithms rely on being provided an initial value, at which value the necessary derivatives are computed to determine the direction (and perhaps distance) to find the next point to evaluate. Depending on the curvature of the function to optimize, and the initial value, the algorithm may get stuck in a local, not global, optimum.

The figure generated below demonstrates this second point. The curve to be maximized has two optima with a saddle point in between. Four possible initial values are shown. When the curve's gradient is evaluated at the two $x_0$ points in red, an optimization algorithm will converge to the local optimum. The two in green will converge to the global optimum. In general, any initial value on the right side of the black vertical line will cause an optimization algorithm to converge to the global optimum; initial values to the left of it will converge to the local optimum.

In [ ]:
# get the data
np.random.seed(42)
X1 = stt.norm(loc=0, scale=1).rvs(500)
X2 = stt.norm(loc=2, scale=0.5).rvs(500)
data = np.r_[X1, X2]

# prep for kde
mn, mx = min(data), max(data)
x= np.linspace(mn, mx, 200)

# kde
kde = stt.gaussian_kde(data, bw_method=0.25)
y = kde(x)

# create the annotation
x1 = -1
y1 = kde(x1)[0]
ann1 = dict(x=x1, y=y1, xref='x1', yref='y1', text='$x_0 = %0.2f\\text{; Gradient →; Local Optimum}$'%x1, showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#ff0000", opacity=0.8,
            font={'color':'#ffffff'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#ff0000")
x2 = 0.40
y2 = kde(x2)[0]
ann2 = dict(x=x2, y=y2, xref='x1', yref='y1', text='$x_0 = %0.2f\\text{; Gradient ←; Local Optimum}$'%x2, showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#ff0000", opacity=0.8,
            font={'color':'#ffffff'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#ff0000")
x3 = 1.5
y3 = kde(x3)[0]
ann3 = dict(x=x3, y=y3, xref='x1', yref='y1', text='$x_0 = %0.2f\\text{; Gradient →; Global Optimum}$'%x3, showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#00FF00", opacity=0.8,
            font={'color':'#000000'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#00FF00")
x4 = 3.0
y4 = kde(x4)[0]
ann4 = dict(x=x4, y=y4, xref='x1', yref='y1', text='$x_0 = %0.2f\\text{; Gradient ; Global Optimum}$'%x4, showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#00FF00", opacity=0.8,
            font={'color':'#000000'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#00FF00")
x5 = 0.78698325
y5 = kde(x5)[0]
ann5 = dict(x=x5, y=y5, xref='x1', yref='y1', text='← Fails; Succeeds →', showarrow=True,
            bordercolor="#c7c7c7", borderwidth=2, borderpad=4, bgcolor="#6d72f1", opacity=0.8,
            font={'color':'#ffffff'}, align="center", arrowhead=2, arrowsize=1, arrowwidth=2,
            arrowcolor="#636363")

# plot
trcs = [go.Scatter(x=x, y=y, mode='lines', name='Kernel Density Estimate'),
        go.Scatter(x=[x5, x5], y=[min(y), max(y)], mode='lines', line={'color':'black'})]
fig = go.Figure(data=trcs, layout=go.Layout(title='Shortfall of Gradient Following'))
anns = list(fig['layout']['annotations'])
anns.extend([ann1, ann2, ann3, ann4, ann5])
fig.update_layout( annotations=anns)
#plyoff.plot(fig, include_mathjax='cdn')
plyoff.iplot(fig)

## Non-Gradent Following Optimization Algorithms
<a id=NGFA></a>
<a href=#top>Go to Top</a>

There are several types of mathematical optimization algorithms which similarly operate on a sequence of individual solutions, but don't use derivatives. I am familiar with [Simulated Annealing](https://en.wikipedia.org/wiki/Simulated_annealing) and [Golden Section Search](https://en.wikipedia.org/wiki/Golden-section_search) opimization.

There is an entirely different class of optimization algorithms which operate on ensembles of solutions - called a *population*, generally iteratively updating them in concert.

## Evolutionary Algorithms
The term [Evolutionary Algorithm (EA)](https://en.wikipedia.org/wiki/Evolutionary_algorithm) refers to a type of population-based metaheuristic optimization algorithm, and is a subset of evolutionary computation. Broadly, evolutionary algorithms are designed around concepts which come from biological evolution. There are several classes of evolutionary algorithms:

- Differential Evolution
- Evolutionary Programming
- Evolutionary Strategy
- Genetic Algorithm
- Genetic Programming ([see here](https://github.com/ahowe42/baseball))
- Learning Classifier System
- Neuroevolution

This set of lectures is focused on the Genetic Algorithm (GA), but could potentially be extended to include [Genetic Programming](https://en.wikipedia.org/wiki/Genetic_programming).

## Other Gradient Eschewing Algorithms
In addition to EA's, there are several other types of metaheuristic optimization algorithms that are based on the idea of optimizing with a population of potential solutions. These include:

- [Ant Colony Optimization - or Traveling Ant Colony Optimization (TACO)](https://en.wikipedia.org/wiki/Ant_colony_optimization_algorithms)
- [Particle Swarm Optimization](https://en.wikipedia.org/wiki/Particle_swarm_optimization)
- [Bees Algorithm](https://en.wikipedia.org/wiki/Bees_algorithm)
- [Adaptive Dimensional Search](https://en.wikipedia.org/wiki/Adaptive_dimensional_search)
- [Gaussian Adaptation](https://en.wikipedia.org/wiki/Gaussian_adaptation)
- Harmony Search - special case of [Evolution Strategy](https://en.wikipedia.org/wiki/Evolution_strategy)

I am only familiar with a few of these.

## Genetic Algorithm
<a id=GA></a>
<a href=#top>Go to Top</a>

The genetic algorithm (GA) is a stochastic population-based metaheuristic optimization algorithm that borrows concepts from biological evolution. In the GA, a solution to an optimization problem is represented as a binary word of length $n$. In the GA parlance, this is metaphorically called an *individual*, or *chromosome*. Each solution is part of an ensemble of solutions which are considered together; this is called the *population*. The function to be optimized is called the *objective function*, or the *fitness function* - the latter is a metaphor to the biological concept of "survival of the fittest".

The semantic meaning of the binary word as a solution that optimizes an objective function (an individual being the most fit in it's environment) depends on the context. For example, if the GA is being used to optimize a real-valued function, the word could be a digital representation of possible real values. The binary word could represent selection flags for a feature selection problem. In [this research article](https://www.researchgate.net/publication/301770005_Regularized_SVM_Classification_with_a_new_Complexity-Driven_Stochastic_Optimizer), I used the GA to simultaneously select a subset of features and one of 9 kernel functions for [kernel SVM classification](https://en.wikipedia.org/wiki/Support-vector_machine). The same binary word had one portion interpreted as binary flags, and the other portion as digital representations of the numbers 1 to 9.

The objective function is usually a map, $\phi\left(X\in\mathcal{R}^{m\times p}, I^n\right) \rightarrow \mathcal{R}$, jointly mapping some data plus a binary word to a scalar real value. GAs have also been developed for multi-objective optimization, in which case the objective function would map to a real-valued vector of length $q$: $\phi\left(X\in\mathcal{R}^{m\times p}, I^n\right) \rightarrow \mathcal{R}^q$. There are several approaches to this - I find approaches that use a [Pareto Front](https://en.wikipedia.org/wiki/Pareto_front) quite appealing.

Almost all optimization algorithms need to start with an initial solution, and the same is true for the GA (and indeed, all Evolutionary Algorithms). For the GA, there may be domain knowledge-based ways to generate an initial population of $P$ individuals. However, it is common to initialize the population randomly, such that each bit has a 50% chance of being turned on.

Starting from the initial population, the GA iterates over a few steps:

1. score the fitness of each individual in the population
2. rank and select individuals for mating
3. mate pairs of individuals to create a new population

    a. crossover
    
    b. mutation
    
    c. GA engineering
4. apply any other GA operators

Each population of individuals is called a *generation*. There are several ways to rank and select individuals for mating. There are also several operators that can be used in the GA, mostly related to creating a new generation.

In [ ]:
# function to print individuals
printIndiv = lambda x: ''.join(['%d'%i for i in x])

In [ ]:
''' generate a sample population with scores '''
# generate the population and scores
n = 10
P = 8
np.random.seed(1906)
population = np.random.rand(P, n) > 0.5
fitness = np.random.rand(P)

# sort
stdIndex = np.argsort(fitness)[::-1]
population = population[stdIndex]
fitness = fitness[stdIndex]

# talk
print('Sample population with %d %d-length individuals'%(P,n))
for indx, (score, individual) in enumerate(zip(fitness, population)):
    print('%d: %s = %0.3f'%(indx, printIndiv(individual), score))

### Mating Ranking & Selection
In the GA, mating is the process of combining two individuals to generate new individuals. I generally have two *parent* solutions mate to create a pair of *child* solutions. I do this is to keep a constant population size, but it's not required. For example, mating could choose to create, based on a randomized choice either:

- a single offspring + new random individual
- a pair of offspring

The new random individual in the first option could be thought of as *adoption*, and would help explore the solution space. Note that I generally use an even population size, as it simplifies this step.

If the GA is being run to optimize multiple objectives, this is the part of the algorithm that is most affected, as fitness scores are predominantly used to determine which individuals mate. I have generally used two approaches to mating ranking & selection - both of the below are more complicated for multi-objective problems.

#### Sorted
In biology, it is often the case that individuals that are similarly fit for their environment mate together (hence, the phrase "she's out of my league"). The metaphor generally holds for the GA, in that solutions with similar objective function scores are mated. This doesn't have to be the case. For example, we could mate pairs of solutions best-fitness to worst-fitness. This approach may help better explore the solution space, but I've never used it. 

The sorted approach to mate ranking and selection is simple, and has the property that each individual mates exactly and only one time. For this, all individuals are sorted according to their fitness scores, then paired off in sequence, so there are $P/2$ mating pairs generated.

#### Roulette
In biology, it is often the case that the most fit individuals mate the most, thus propagating their successful genes. This metaphor holds by design with the roulette method for generating mating pairs.

The roulette method starts by sorting all individuals according to their fitness scores, then generating a biased roulette bar, in which the individual bins are of gradually decreasing size as computed and visualized (for $P=4$) here:
\begin{equation}
b_i = \frac{2i}{n\left(n+1\right)}, i=1,\ldots,P\text{.}
\end{equation}

<center><img src='../images/roulette_selection.png' width='400' height='200'></center>

When the cumulative sum of these bin widths is computed, we get upper bounds for the roulette bins, which completely partition the $[0, 1]$ interval. Each bin corresponds to an individual in the population; since the population was already sorted by fitness score, the wider bins correspond to the most fit individuals.

Since we don't want the population to grow, we will generate $P/2$ mating pairs. To do so, $P$ random numbers are generated uniformly from $[0, 1]$ ($U\left(0, 1\right)$) and placed in the appropriate bin. For each random variate in the $i^\text{th}$ bin, the corresponding individual will be selected to mate. In this way, individuals with a better fitness score are overrepresented in the mating pool. The last step is to randomly permute the ordering of the individuals in the mating pool.

Note that this means that, while the most fit individuals will tend to mate the most frequently, they won't just mate with individuals with similar fitness scores.

In [ ]:
''' pair individuals for mating - sorted method '''
# skipping sorting, as the population is already sorted
pairs = np.reshape(range(P), (P//2, 2))
# talk
for (indx, pair) in enumerate(pairs):
    print('Mating pair %d: %s (%0.3f) <-> %s (%0.3f)'%(indx, printIndiv(population[pair[0]]), fitness[pair[0]],
                                                       printIndiv(population[pair[1]]), fitness[pair[1]]))

In [ ]:
''' pair individuals for mating - roulette method '''
# generate the bounds and show the bins
binUBounds = np.cumsum(2*np.linspace(P, 1, P)/(P*(P + 1.0)))
bnds = [0] + binUBounds.tolist()
print('Roulette bins: %r'%[('%0.2f'%f, '%0.2f'%t) for f, t in zip(bnds[:-1], bnds[1:])])

# generate mating frequencies
np.random.seed(2022)
rands_in_bins = np.repeat(np.random.rand(P), P) >= np.tile(binUBounds, P)
pairs = np.reshape(np.random.permutation(np.sum(np.reshape(rands_in_bins, [P]*2), axis=1)), (P//2, 2))

# talk
for (indx, pair) in enumerate(pairs):
    print('Mating pair %d: %s (%0.3f) <-> %s (%0.3f)'%(indx, printIndiv(population[pair[0]]), fitness[pair[0]],
                                                       printIndiv(population[pair[1]]), fitness[pair[1]]))

### Crossover
In biology, when a pair of individuals mate, their chromosomes are combined in a process called [chromosomal crossover](https://en.wikipedia.org/wiki/Chromosomal_crossover). An illustration from 1916 by researcher Thomus Hunt Morgan demonstrates the creation of recombinant genes through this process.

<img src='https://upload.wikimedia.org/wikipedia/commons/0/0e/Morgan_crossover_1.jpg' height='500' width='500'>

In the GA, after mating pairs are generated from a population, whether or not the crossover operation is employed is controlled by flipping a coin, biased according to the crossover probability $P_X$. If a random variate generated from $U\left(0, 1\right)$ is less than $P_X$, crossover is used. Otherwise, a mating pair produces a pair of offspring that are genetic replicants. Since crossover increases exploration of the objective function's solution space, I generally use a high probability - at least $P_X = 0.7$. I use three types of crossover.

#### Single-Point
In single-point crossover, a single crossover point (see what we did there?) is determined by generating a random value from $U\left(1, n\right)$. The two individuals in a mating pair have their chromosomes traded at that point, as shown in the illustration above.

#### Dual-Point
Dual-point crossover works similarly, except two crossover points are generated randomly in the same way as single-point crossover. The chromosomes are partitioned and traded at these points. This is shown in the illustration below, also from Thomas Hunt Morgan.

<img src='https://upload.wikimedia.org/wikipedia/commons/4/45/Morgan_crossover_2.jpg' width='500' height='500'>

#### Uniform
Uniform crossover is the logical endpoint of the sequence of types of crossovers shown here. With uniform crossover, a random variate is generated from $U\left(0, 1\right)$ for each of the $n$ elements in the chromosomes being crossed over. For each element such that the random value is less than $P_X$, that element of the binary word is traded between the two individuals in the mating pair.

In [ ]:
''' single-point crossover '''
# generate the crossover flags
np.random.seed(42)
probXover = 0.7
xovers = np.random.rand(len(pairs)) < probXover

# crossover
newPop = [None]*(2*len(pairs))
for (indx, pair) in enumerate(pairs):
    # get the individuals
    p1 = population[pair[0]]
    p2 = population[pair[1]]
    if xovers[indx]:
        # genetic replication
        xoverpoint = 0
        n1, n2 = p1, p2
        newPop[indx*2] = n1
        newPop[indx*2+1] = n2
    else:
        # crossover
        xoverpoint = np.random.randint(1, n-1)
        n1 = np.concatenate((p1[:xoverpoint], p2[xoverpoint:]))
        n2 = np.concatenate((p2[:xoverpoint], p1[xoverpoint:]))
        newPop[indx*2] = n1
        newPop[indx*2+1] = n2
    # talk
    print('Mating pair %d (xover=%d): %s <-> %s --> %s & %s'%(indx, xoverpoint, printIndiv(p1), printIndiv(p2), printIndiv(n1), printIndiv(n2)))
newPop = np.array(newPop)

In [ ]:
''' dual-point crossover '''
# generate the crossover flags
np.random.seed(42)
probXover = 0.7
xovers = np.random.rand(len(pairs)) < probXover

# crossover
newPop = [None]*(2*len(pairs))
for (indx, pair) in enumerate(pairs):
    # get the individuals
    p1 = population[pair[0]]
    p2 = population[pair[1]]
    if xovers[indx]:
        # genetic replication
        xoverpoints = [0,0]
        n1, n2 = p1, p2
        newPop[indx*2] = n1
        newPop[indx*2+1] = n2
    else:
        # crossover
        xoverpoints = np.sort(np.random.permutation(n-1)[:2]+1)
        n1 = np.concatenate((p1[:xoverpoints[0]], p2[xoverpoints[0]:xoverpoints[1]], p1[xoverpoints[1]:]))
        n2 = np.concatenate((p2[:xoverpoints[0]], p1[xoverpoints[0]:xoverpoints[1]], p2[xoverpoints[1]:]))
        newPop[indx*2] = n1
        newPop[indx*2+1] = n2
    # talk
    print('Mating pair %d (xover=[%d, %d]): %s <-> %s --> %s & %s'%(indx, xoverpoints[0], xoverpoints[1], printIndiv(p1), printIndiv(p2), printIndiv(n1), printIndiv(n2)))
newPop = np.array(newPop)

In [ ]:
''' uniform-point crossover '''
# generate the crossover flags
np.random.seed(1211)
probXover = 0.7
xovers = np.random.rand(len(pairs)) < probXover

# crossover
newPop = [None]*(2*len(pairs))
for (indx, pair) in enumerate(pairs):
    # get the individuals
    p1 = population[pair[0]]
    p2 = population[pair[1]]
    if xovers[indx]:
        # genetic replication
        xoverpoint = [0]*n
        n1, n2 = p1, p2
        newPop[indx*2] = n1
        newPop[indx*2+1] = n2
    else:
        # crossover
        xoverpoints = probXover > np.random.rand(n)
        n1 = p1*xoverpoints + p2*~xoverpoints
        n2 = p1*~xoverpoints + p2*xoverpoints
        newPop[indx*2] = n1
        newPop[indx*2+1] = n2
    # talk
    print('Mating pair %d (xover=%s): %s <-> %s --> %s & %s'%(indx, printIndiv(xoverpoints), printIndiv(p1), printIndiv(p2), printIndiv(n1), printIndiv(n2)))
newPop = np.array(newPop)

### Mutation
After mating pairs use crossover to generate a new population of individuals, the new individuals' chromosomes are mutated. Mutation simply entails flipping bits in the chromosome randomly, according to a small mutation probability $P_M$. To mutate an individual, $n$ random variates are generated from $U\left(0, 1\right)$. For any that are less than $P_M$, the corresponding bits are flipped. I generally use a low probability, such as $P_M=0.10$.

In [ ]:
''' mutation '''
# generate mutation flags
probMutate = 0.1
mutators = probMutate > np.random.rand(P, n)
dat = pd.DataFrame(data = [printIndiv(m) for m in mutators], columns=['Mutate Flags'])

# mutate
dat['Before'] = [printIndiv(n) for n in newPop]
newPop = newPop.copy()
newPop[mutators] = ~(newPop[mutators])
dat['After'] = [printIndiv(n) for n in newPop]

# talk
display(dat)

### GA Engineering
One criticism of the GA is that, due to its stochastic nature, subsequent replications of the GA can end up with very different solutions to the same problem. This typically only occurs if a problem is very large and / or when the number of generations and population size are set too low. One solution is to employ an operator called *GA engineering*, which is intended to reduce variability between replications. Continuing the metaphor, GA engineering is akin to genetic engineering, in which genes from a more successful individual - such as a plant that is pest-resistant - is inserted into other individuals.

GA engineering is employed after a new population has been created, when the best solution from the previous generation is better than the best solution of the current generation. The process takes the best individual from the previous and current generations, and finds the differences in their chromosomes; we'll call $\Delta_{P-C}$ the bits that are on in the previous but not current best chromosome. $P$ random values are then generated from $U\left(0, 1\right)$; for individuals in the new population corresponding with a random variate being less than the probability of engineering $P_E$, the bits in $\Delta_{P-C}$ are inserted. I usually set $P_M > P_E > P_X$.

In [ ]:
''' GA engineering '''
engineer_rate = 0.2
# make a previous best
prevBest = np.array([True, True, False,  True, True,  False, False, False, True, True])
# get the difference
currBest = population[np.argmax(fitness),:]
diffLocs = np.logical_xor(currBest, prevBest)
diffVals = prevBest[diffLocs]

# print bests
print('Previous Best: %s'%printIndiv(prevBest))
print(' Current Best: %s'%printIndiv(currBest))

# coin flips for implementing
np.random.seed(42)
engme = engineer_rate > np.random.rand(P)

# engineer
dat = pd.DataFrame({'Engineer':engme, 'Before':[printIndiv(m) for m in newPop]})
newPopC = np.zeros((P, n), dtype=bool)
rows = np.arange(P)
# first build the output by adding the population elements that won't be changed
rows_nochg = rows[~engme]
newPopC[rows_nochg, :] = newPop[~engme, :]
# now edit the rest of the population
jnk = newPop[engme, :]
jnk[:, diffLocs] = diffVals
newPopC[rows[engme], :] = jnk
newPop = newPopC
dat['After'] = [printIndiv(m) for m in newPop]

# talk
display(dat)

### Elitism
In the GA, individuals in a generation generally die after mating, leaving the next generation all offspring. In real life, it can occasionally happen that an especially fit individual will remain a desirable mating partner for more than one generation; think Sean Connery, Harrison Ford, or Charlton Heston. This cross-generation interaction can be implemented as an option in the GA, with the *elitism* rule. When elitism is on the most fit solution from the current generation does not die, but joins the mating pool in the next generation. If there is already an individual in the next generation with the same chromosome, this has no effect.

An advantage of elitism is that every generation always has the best individual in the population, so the GA's performance is monotonic. However, elitism means the population may grow with each generation. This can complicate mate ranking & selection, but is easily handled. Alternatively, we could design the GA to drop a random individual from the population before inserting the best individual, but this would hinder exploration of the solution space.

Since elitism means that the best solution ever found by the GA is always in the mating pool, GA engineering is of limited value. I don't use both together.

In [ ]:
''' elitism '''
varis = np.arange(n) + 1
bin_to_dec = 2**(varis - 1)
    
# check if best is currently in new_pop
tmp1 = np.sum(newPop*bin_to_dec, axis=1)
tmp2 = np.sum(currBest*bin_to_dec)

if tmp2 not in tmp1:
    population = np.vstack((newPop, currBest))
    print('Elitism applied')
    for (indx, individual) in enumerate(population):
        print('%d: %s'%(indx, printIndiv(individual)))
else:
    population = newPop.copy()
    print('Elitism not necessary')

These steps are iterated over a specified number of generations.

While the GA is not guaranteed to find the global optima, with appropriately-set parameters, it will most likely get very close. Indeed, the [no free lunch theorem](https://en.wikipedia.org/wiki/No_free_lunch_theorem) tells us that no optimization algorithm can outperform all others on all problems. Wolpert and Macready, who introduced the theorem stated "*We have dubbed the associated results NFL theorems because they demonstrate that if an algorithm performs well on a certain class of problems then it necessarily pays for that with degraded performance on the set of all remaining problems.*".

Optimization problems have two jobs:
- exploration: As compared to gradient-followers, or other otimization algorithms which consider solutions individually, population-based algorithms do a much better job of exploring the solution space. In the GA specifically, operators which can generate novel solutions such as mutation and crossover, help explore the solution space. A best solution which is near a local optimum may be a single bit's mutation away from the global optimum.

- exploitation: The second job of an optimization algorithm is to more carefully explore promising areas of the solution space. This is something some gradient-following algorithms do well *when they are near the global optimum*. The BFGS algorithm is one. In the GA, GA engineering and the elitism rule help exploit good solutions, as they ensure useful elements of good chromosomes are not lost through mutation or crossover.

In some cases, it may be beneficial for the GA to explore more in early generations, and exploit more in later generations. This could be implemented by setting a higher initial mutation rate, then allow the crossover and mutation probabilities to decay by generation. This is similar to the adaptive learning rate used by many optimizers for training deep learning neural networks. In simulated annealing, a decaying temperature parameter allows the algorithm to explore the solution space more thoroughly in earlier iterations. When the temperature is hot, the algorithm has a probability to make a move to a "worse" solution. As the temperture cools, the probability of a bad move approaches 0.

# Genetic Algorithm Applied
We demonstrate use of the GA on three problems:

- Feature Selection for Machine Learning
- Best-fitting Distribution Selection
- Multivariate Function Minimization

## Feature Selection for Machine Learning

<a id=FSML></a>
<a href=#top>Go to Top</a>

Statistical modelers have been trying models on subsets of features for almost as long as statistical modeling (most of what we call "machine learning" is actually statistical modeling) has been around. Perhaps unimaginably, we call the process of selecting a subset of available features feature selection. In feature selection, we use some procedure to generate subsets of the existing features, fit a model to them, and evaluate that model to find an optimal subset. The goal of feature selection is usually to balance two considerations: model performance and model complexity. It is generally beneficial for a model to be simpler - to use fewer features, for example. Practitioners often prefer a simpler model, even if it performs slightly worse than a more complex model. This follows the principle of [occam's razor](https://en.wikipedia.org/wiki/Occam%27s_razor).

A simple way to perform feature selection, that guarantees finding the most optimal subset of features, is combinatorial enumeration - a.k.a. brute force. Combinatorial enumeration does exactly what it sounds like - the model is evauated on the enumeration of all possible combinations of features. This is no mean feat, as the number of ways to combine $p$ features is exponential in $p$; there are $2^{p-1}$ possible subsets. The GA is a useful tool for feature selection; for $p$ features, each individual is a p-length binary word indicating that a feature is in that solution (1) or out of it (0). If $p=8$, for example, one solution may be $10011001$; in this case, features 1,4,5,8 will be used, while 2,3,6,7 will not.

The [GA Feature Selection Notebook](./GA_FeatureSelection.ipynb) demonstrates feature selection with the GA, using some data simulated with a known dependence structure. It begins with simulating $n=100$ observations of $p=20$ independent features, with each value drawn randomly from $U\left(0, \gamma=5\right)$, so $X\in\mathcal{R}^{n\times p}$. A target variable $y$ is then generated as
\begin{equation}
y = 8X_0 - X_1 + 4X_2
\end{equation}
Most of the features are uncorrelated with the target and each other, as shown in the [Feature Correlations Plot](https://github.com/ahowe42/FeatureCorrelationsPlot):

<center><img src='../images/Correlations_8X0__1X1_4X2.png' height='1600'></center>

With $p=20$ features, there are a total of $2^{20}-1=1,048,575$ possible subset regression models which can be fit to these features. 

The GA was run with the inputs shown below, terminating early after around 90 generations, and evaluating only $25,828$ subsets of features - 2.5% of the total possible.

<center><img src='../images/GAProgress_20220131161948_8X0__1X1_4X2_RegressionMetric.png' height='2000'></center>

The top pane of the progress plot shows that the GA found the final solution in the 11$^{\text{th}}$ generation - the solution includes the three correct features + only one additional feature. The annotation on the plot indicates the solution binary word, number features included, subset fitness score, and the score relative to the score of the saturated model (all features included). The bottom pane plots the average fitness score of all solutions per generation. Note that, even while the best solution was found in an early generation, the GA was still exploring the solution space quite well, as the average scores remained quite high relative to the best score, oscillating around 6.25 or so.

Finally, I generated four typical regression diagnostic plots based on the model residuals (from top left, clockwise):

- target vs. predictions scatter plot - to assess how well the predictions match
- histogram of residuals - to assess if the residuals appear to be Gaussian white noise centered on $0$
- residuals ordered by data - to assess if there are any sequence-dependent patterns
- residuals by target - to assess if the level of residuals shows a pattern vis-a-vis the level of target values

<center><img src='../images/GAPerformance_20220131162231_8X0__1X1_4X2_RegressionMetric_metric__RMSE___estim_LinearRegression_fit_intercept_False___optimGoal__1.png' height='2000'></center>

The plots show that the predicted values match the targets quite well, with very small errors that have no perceivable pattern.

In conclusion, the GA did a good job of finding very close to the true [data generating process](https://en.wikipedia.org/wiki/Data_generating_process), settling on a set of features that included only a single extraneous feature. This performance is very satisfying, but recall that the data was generated with no additional noise. Adding noise would have changed the outcome, but this is likely an artifact of the simulation protocol, in that every feature was essentially white noise. This demonstration used a linear regression model, but other machine learning models could have been passed to the objective function. In fact, something completely different could have been used for the objective function. One good possibility could be some function of the mutual information among pairs of included features and the target variable. Finally, note that feature selection is particularly amenable to multi-objective optimization, in that we'd like to simultaneous find the best-fitting model, while penalizing for too much complexity. In this case, complexity could simply be measured by how many features are included.

## Best-fitting Distribution Selection

<a id=BFDS></a>
<a href=#top>Go to Top</a>

## Multivariate Function Minimization

<a id=MFM></a>
<a href=#top>Go to Top</a>

<a href=#top>Go to Top</a>